In [ ]:
## Coding a single neuron with inputs and weights
def neuron(inputs, weights, bias):
    sum = 0
    for pair in zip(inputs, weights):
        a, b = pair
        sum += a * b

    sum += bias

    return sum 

In [ ]:
inputs = [1, 2, 3]
weights = [0.2, 0.8, -0.5] ## correspond to the importance of each input

print(neuron(inputs, weights, 2))

In [ ]:
## implementing a layer of neurons 
def layer(inputs, weights, bias):
    output = []
    for i in range(len(weights)):
        sum = 0
        for j in range(len(weights[i])):
            sum += weights[i][j] * inputs[j]
        sum += bias[i]
        output.append(sum)
    return output
        

In [ ]:
weights = [[0.5, 0.5, 0.5], [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]]

print(layer(inputs,weights, [0, 0, 1])) 

In [ ]:
## Coding with numpy

import numpy as np

In [ ]:
# Represents a single neuron
bias = 2
outputs = np.dot(inputs, weights) + bias
print(outputs)

In [ ]:
# Coding a layer

weights = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
inputs = [1, 2, 3]
bias = [1, 1, 1]

outputs = np.dot(weights, inputs) + bias
print(outputs.reshape(3, 1))

In [ ]:
## Implementing batches of data
batch_data = [[1, 2, 3], [2, 2, 2], [1, 5, 4], [6, 4, 1]]
weights = [[1, 1, 1], [1, 1, 1], [1, 1, 1]]
bias = [2, 2, 2]

outputs = np.dot(batch_data, np.array(weights).T) + bias
print(outputs.shape)

## Shape of the Output must be (4, 3), where we have 4 batches of data and 3 outputs for each of the 3 neurons

In [ ]:
## creating multiple layers - In this specific instance, we'll take 2 layers with 3 batches of data

weights_2 = [[1, 1, 1], [1, 1, 1], [1, 1, 1]]
bias_2 = [2, 2, 2]

final_output = np.dot(outputs, np.array(weights_2).T) + bias

print(final_output.shape) ## For all batches of data, I need to have an output from each neuron in the layer -> shape = (no. of samples, no.of neurons)

In [ ]:
## generating data - spiral 
from nnfs.datasets import spiral_data
import numpy as np
import nnfs

nnfs.init()
import matplotlib.pyplot as plt 
X, y = spiral_data(samples = 100, classes = 3) # X- data , y - class
# plt.scatter(X[:, 0], X[:, 1])
# plt.show()
print(X)

In [ ]:
## seeing the different classes
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='brg')
plt.show()

In [7]:
## creating a class for Dense Neural layers
## It takes the numer of inputs and the number of neurons for each layer
import numpy as np
class Layer_Dense:
    def __init__(self, num_features, num_neurons):
        self.weights = 0.01 * np.random.randn(num_features, num_neurons)
        self.bias = np.zeros((1, num_neurons))
        
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.bias


In [ ]:
dense1 = Layer_Dense(2, 3)
dense1.forward(X)
print(dense1.output[:5])

In [1]:
## learning the different sum commands in python 
import numpy as np
A = [[1, 2, 3], [4, 5 , 6], [7,8, 9]]
np.sum(A, axis = 0, keepdims=True) 

array([[12, 15, 18]])

In [ ]:
### Learning about broadcasting rules - Just check each dimension value, if it differs, then check if one of them is none or zero. If it is, then it is broadcastable.
### The way we broadcast is to extend it along the dimension that it is short in. 
out = np.array(A) + np.array([[1, 2, 3]]).T

print(np.array(out))

## Practice - From each row in a 2x2 grid, remove the maximun element from each element of that row 

matrix = np.array([
    [1, 5, 3],
    [7, 2, 9],
    [4, 6, 8]
])

max_in_rows = np.amax(matrix, axis=1, keepdims=True)
out = matrix - max_in_rows
print(out)

In [ ]:
### Activation functions

### The need of activation functions is mainly to represent the non-linearity in the data - This makes my network more expressive

### Different kinds of activation functions: 
### 1.) RELU (Rectified Linear Unit) : It is a function that just takes the maximum between 0 and x (Different variants available)
### 2.) Sigmoid Activation : Used to truncate the output between 0 and 1
### 3.) Softmax Activation Function: Used in multi-class classification



In [5]:
### RELU Class
class RELU: 
    def forward(self, inputs): 
        self.output = np.maximum(0, inputs) ### Output will have the same shape as the input

In [3]:
### Softmax Activation Function
import numpy as np
class Softmax: 
    def forward(self, inputs): 
        ## Find the raw exponentials
        raw_exp = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        ## Find the probabilistic measure 
        self.output = raw_exp / np.sum(raw_exp, axis = 1, keepdims=True)

In [ ]:
### Coding a Forward Pass (Without Loss Function)

### Steps: 
# 1) Define the number of layers you need in the networks along with the number of features in the feature set, also generate the data
# 2) Now, you have to use the activation functions on the outputs of the Neurons - RELU, then Softmax
# 3) Return the output from the Softmax function 

# generate the spiral data
## generating data - spiral 
from nnfs.datasets import spiral_data
import numpy as np
import nnfs

nnfs.init()
import matplotlib.pyplot as plt 
X, y = spiral_data(samples = 100, classes = 3) # X- data , y - class

## Create the first Layer with 3 neurons
Layer1 = Layer_Dense(2, 3)

## Get the output from the first layer
Layer1.forward(X) ## 300 x 3

## Send he output from the first layer to the activation function 
relu_activator = RELU()
relu_activator.forward(Layer1.output)

## This now forms the input to the second Neural Network
Layer2 = Layer_Dense(3, 3)

## Generate the output from the second layer using the relu output
Layer2.forward(relu_activator.output)

## Finally, send this output to the softmax function
softmax_activator = Softmax()
softmax_activator.forward(Layer2.output)

## Print the output for each of the data points
print(softmax_activator.output)

In [35]:
## Now, we need a way to evaluate our predictions and tune our weights accordingly
## Loss Functions 
# 1) Cross Entropy Loss - helps to analyze the predictions made during the classification problem = - Summation(True Label * log(predicted label))
## Side Note: One-hot encoding -- This technique is used for representing categorical features, like the colors red, blue, green could be represented as [1, 0, 0] if the output is red

Softmax_outputs = np.random.randn(3, 3)
obj = Softmax()
obj.forward(Softmax_outputs)
check = obj.output

class_target = [0, 1, 1]
print(check[[0, 1, 2], class_target])

# find the negative log of all the target class outputs
neg_log = - np.log(check[range(len(check)), class_target])
print(neg_log)

## find the mean of all the data points to finally return the total loss
avg = np.mean(neg_log)
print(avg)

[0.12300237 0.37120515 0.26867129]
[2.09555162 0.9910004  1.31426661]
1.4669395423242937


In [45]:
## Finding the output of the element wise multiplication
y_true = np.array([
    [1, 0, 0], 
    [0, 1, 0], 
    [0, 1, 0]
])

y_pred = check

a = y_true * y_pred
b = np.sum(a, axis = 1)
c = - np.log(b)

print(np.mean(c))


1.4669395423242937


In [79]:
class Loss:
    def __init__(self):
        pass

    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

class Loss_CategoricalEntropy(Loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)

        # Clip predictions to avoid log(0) errors
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        if len(y_true.shape) == 1:
            # Fix: Use samples instead of len(samples)
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Negative log likelihood
        negative_log = -np.log(correct_confidences)
        return negative_log


In [91]:
## Coding forward pass from scratch 

## Data 
from nnfs.datasets import spiral_data
import numpy as np
import nnfs

nnfs.init()
import matplotlib.pyplot as plt 
X, y = spiral_data(samples = 100, classes = 3) # X- data , y - true class

## Set up the network
Layer_1 = Layer_Dense(2, 3)
Layer_2 = Layer_Dense(3, 3)

## Set up the peripheries - Softmax and categorical loss
softmax = Softmax()
relu = RELU()
Categorical = Loss_CategoricalEntropy()

Layer_1.forward(X)
relu.forward(Layer_1.output)

Layer_2.forward(relu.output)
softmax.forward(Layer_2.output)

loss = Categorical.calculate(softmax.output, y)
print(loss)


1.0986104


In [93]:
## Accuracy 
Softmax_outputs = np.random.randn(3, 3)
obj = Softmax()
obj.forward(Softmax_outputs)
check = obj.output

class_target = [0, 1, 1]

predictions = np.argmax(check, axis = 1)

if (len(class_target) == 2):
    class_target = np.argmax(class_target, axis = 1)

accuracy = np.mean(predictions == class_target)
print(accuracy)

0.6666666666666666


In [ ]:
## How do I update these weights and biases effectively. 2 naive solutions: random selection and update
## Better Solution: Gradient Descent 

